In [2]:
%cd LLaMA-Factory
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

/usr/local/lib/python3.10/dist-packages/IPython/core/magics/osm.py:393: UserWarning: using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})


[Errno 2] No such file or directory: 'LLaMA-Factory'
/workspace/LLaMA-Factory


## 학습전 모델

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('meta-llama/Llama-3.2-3B-Instruct')
model = AutoModelForCausalLM.from_pretrained('meta-llama/Llama-3.2-3B-Instruct')
model = torch.nn.DataParallel(model).cuda()

In [3]:
##학습후 모델

In [4]:
tokenizer = AutoTokenizer.from_pretrained('./output_dir')
model = AutoModelForCausalLM.from_pretrained('./output_dir')
model = torch.nn.DataParallel(model).cuda()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [10]:
device = "cuda:0"

messages = [
    {"role": "system", "content": "당신은 회사명을 찾는 챗봇입니다. \n\n ## 지시 사항 ## \n1. 주어진 입력에 나와있는 회사명을 찾으세요 \n2. 회사명을 찾을수 없다면, 빈 파이썬 리스트를 출력하고, 임의로 답변하려고 시도하지 마세요. \n3. 출력은 반드시 파이썬의 리스트 형태로 생성하십시오.\n4. 뉴스데이터:이 주어지면 회사리스트: 다음에 파이썬 리스트 형태로 작성하세요."},
    {"role": "user", "content": "뉴스데이터: 인공지능(AI) 반도체 시장의 흐름이 ‘맞춤형’ 시대로 접어들면서 메모리 시장에도 지각변동이 일어날 것으로 전망된다. 수년간 AI 메모리 업계를 지배했던 고대역폭메모리(HBM) 외에도 다양한 D램이 차세대 AI 칩으로 거론되기 시작하면서 SK하이닉스의 시장 독주에도 균열이 생길 수 있다.2일 업계에 따르면 엔비디아가 독주하던 AI 반도체 시장에서 새로운 변화가 일어난 것은 ‘추론’ 시장이 고개를 들고 있기 때문이다. AI 반도체는 크게 학습용과 추론용으로 나뉜다. 학습용 반도체는 거대언어모델(LLM) 개발을 위해 방대한 데이터를 단순한 모델로 만들 때 활용된다."}
]

encodeds = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt")
model_inputs = encodeds.to(device)

In [11]:
input_text = tokenizer.decode(model_inputs[0])+ '회사리스트 :'

In [14]:
inputs = tokenizer(input_text, return_tensors="pt")
eos_token_id = tokenizer.convert_tokens_to_ids("<|eot_id|>")

with torch.no_grad():
    outputs = model.module.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=512, eos_token_id=eos_token_id)
    print(tokenizer.decode(outputs[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


<|begin_of_text|><|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 02 Jan 2025

당신은 회사명을 찾는 챗봇입니다. 

 ## 지시 사항 ## 
1. 주어진 입력에 나와있는 회사명을 찾으세요 
2. 회사명을 찾을수 없다면, 빈 파이썬 리스트를 출력하고, 임의로 답변하려고 시도하지 마세요. 
3. 출력은 반드시 파이썬의 리스트 형태로 생성하십시오.
4. 뉴스데이터:이 주어지면 회사리스트: 다음에 파이썬 리스트 형태로 작성하세요.<|eot_id|><|start_header_id|>user<|end_header_id|>

뉴스데이터: 인공지능(AI) 반도체 시장의 흐름이 ‘맞춤형’ 시대로 접어들면서 메모리 시장에도 지각변동이 일어날 것으로 전망된다. 수년간 AI 메모리 업계를 지배했던 고대역폭메모리(HBM) 외에도 다양한 D램이 차세대 AI 칩으로 거론되기 시작하면서 SK하이닉스의 시장 독주에도 균열이 생길 수 있다.2일 업계에 따르면 엔비디아가 독주하던 AI 반도체 시장에서 새로운 변화가 일어난 것은 ‘추론’ 시장이 고개를 들고 있기 때문이다. AI 반도체는 크게 학습용과 추론용으로 나뉜다. 학습용 반도체는 거대언어모델(LLM) 개발을 위해 방대한 데이터를 단순한 모델로 만들 때 활용된다.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

회사리스트 : ['SK하이닉스', '엔비디아']<|eot_id|>


In [15]:
from huggingface_hub import HfApi
api = HfApi()
username = "irene93"

MODEL_NAME = 'Llama-3-Company-Extract'
api.create_repo(
    token=### 토큰값 ,
    repo_id=f"{username}/{MODEL_NAME}",
    repo_type="model"
)

api.upload_folder(
    token=### 토큰값 ,
    repo_id=f"{username}/{MODEL_NAME}",
    folder_path="output_dir",
)

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/irene93/Llama-3-Company-Extract/commit/b6b6327b940117d49fbe289c8d6fd1dc8a6d6ed0', commit_message='Upload folder using huggingface_hub', commit_description='', oid='b6b6327b940117d49fbe289c8d6fd1dc8a6d6ed0', pr_url=None, repo_url=RepoUrl('https://huggingface.co/irene93/Llama-3-Company-Extract', endpoint='https://huggingface.co', repo_type='model', repo_id='irene93/Llama-3-Company-Extract'), pr_revision=None, pr_num=None)